But it's so boring. Also it's not full automated way.

Unfortunately, pd.read_html and requests + beautifulsoup do not allow parsing info.
Therefore, I decided to use API.

First and foremost, I registered in https://etherscan.io/ to get API key.

FREE API PLAN allows doing 5 calls per second. It's appropriate number for our goals.

API-Key is kept in .env file.But it's so boring. Also it's not full automated way.

Unfortunately, pd.read_html and requests + beautifulsoup do not allow parsing info.
Therefore, I decided to use API.

First and foremost, I registered in https://etherscan.io/ to get API key.

FREE API PLAN allows doing 5 calls per second. It's appropriate number for our goals.

API-Key is kept in .env file.

In [7]:
import requests
from bs4 import BeautifulSoup
from application.load_transaction_data import load_data
df = load_data()
token_transactions_df = df[df[['Value_IN(ETH)', 'Value_OUT(ETH)']].sum(axis=1) == 0]
ether_transactions_df = df[df[['Value_IN(ETH)', 'Value_OUT(ETH)']].sum(axis=1) > 0]

In [5]:
token_transactions_df = df[df[['Value_IN(ETH)', 'Value_OUT(ETH)']].sum(axis=1) == 0]

In [8]:
token_transactions_df

,Txhash,Blockno,DateTime,From,To,Value_IN(ETH),Value_OUT(ETH),TxnFee(USD),Historical $Price/Eth,Status,ErrCode,Method
2530,0x2394fdb6228d761f899156ebbc06554d0701c5952500...,14644335,2022-04-24 00:40:23,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xf403c135812408bfbe8713b5a23a04b3d48aae31,0.0,0.0,35.279306,2922.37,NaN,NaN,Deposit All
2529,0x167c46dd943aa9ff360ff99fbf2683fc6e78c377f472...,14644327,2022-04-24 00:38:04,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x2dded6da1bf5dbdf597c45fcfaa3194e53ecfeaf,0.0,0.0,21.146652,2922.37,NaN,NaN,Add_liquidity
2528,0xbc81557cb35bb4eda725803c320eee011b06e92a05af...,14644319,2022-04-24 00:37:11,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0.0,0.0,37.972472,2922.37,NaN,NaN,0x415565b0
2527,0x2e2926cccdc525856f0aa64e70917ffbec4a02f557f1...,14644315,2022-04-24 00:36:21,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9,0.0,0.0,33.513294,2922.37,NaN,NaN,Borrow
2525,0xb17a1d2d00e3354bc3f8a522780f30f3c7b89c5c57af...,14644314,2022-04-24 00:36:05,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xccf4429db6322d5c611ee964527d42e5d685dd6a,0.0,0.0,11.859852,2922.37,NaN,NaN,Mint
...,...,...,...,...,...,...,...,...,...,...,...,...
9,0xe426ef6d96afe8c676d644e0b008bc038478f4c89d6a...,10601646,2020-08-05 19:31:06,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.0,0.0,4.266532,401.12,NaN,NaN,Approve
8,0xdbb02f4996966d81b0935938bfa7a751b53b37d68b5e...,10601645,2020-08-05 19:31:01,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,0.0,0.0,4.544130,401.12,NaN,NaN,Approve
7,0xb550117ab3e407bc203ad24ae42a64a16971e833f9ee...,10601645,2020-08-05 19:31:01,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0.0,0.0,4.363306,401.12,NaN,NaN,Approve
6,0x720f6a4d8999ae940cb2df3a633034ea9ca1078bef97...,10601597,2020-08-05 19:19:02,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,0x4678f0a6958e4d2bc4f1baf7bc52e8f3564f3fe4,0.0,0.0,67.162914,401.12,NaN,NaN,Build


In [13]:
import pandas as pd
import re
from tqdm import tqdm

def extract_token_info(token_url):
    result_dict = {}
    try:
        result_dict['from'] =  token_url.find("span", class_='hash-tag text-truncate hash-tag-custom-from tooltip-address').text
    except Exception as e:
        result_dict['from'] = e
    try:
        result_dict['to'] = token_url.find("span", class_='hash-tag text-truncate hash-tag-custom-to tooltip-address').text
    except Exception as e:
        result_dict['to'] = e
    try:
        token1_amount = token_url.find("span", 'data-toggle'=='tooltip').text
        result_dict['amount'] = float(token1_amount.replace(",", "_"))
    except Exception as e:
        result_dict['amount'] = e
    try:
        # token1_usd = token_url.find("button", class_='u-label u-label--value u-label--primary rounded mb-n1 ml-1').text
        token1_usd = re.findall(r"\(\$.*?\)", str(token_url))[0]
        token1_usd = token1_usd.strip("()").replace(" (", "").replace("$", "")
        result_dict['usd'] = float(token1_usd.replace(",", "_"))
    except Exception as e:
        result_dict['usd'] = e
    try:
        result_dict['token_hash'] = token_url.find("a").get("href").split("?")[0].split("/")[2]
    except Exception as e:
        result_dict['token_hash'] = e
    return result_dict

def extract_tokens_info(transaction_hash):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    req = requests.get(f"https://etherscan.io/tx/{transaction_hash}",
                   headers=headers)
    content = req.content
    soup = BeautifulSoup(content)

    tokens_df = []
    bad_tokens = []
    errors = []
    all_tokens = soup.find_all("li", class_='media align-items-baseline mb-2')
    for index, tokens in enumerate(all_tokens):
        try:
            tokens_df.append(
                pd.DataFrame.from_dict(extract_token_info(all_tokens[index]), orient='index').T
            )
        except Exception as e:
            bad_tokens.append(index)
            errors.append(e)
    try:

        tokens_df = pd.concat(tokens_df)
    except ValueError:
        tokens_df = pd.DataFrame(data=[transaction_hash], columns=['Txhash'])
    tokens_df['Txhash'] = transaction_hash
    return tokens_df

all_tokens_df = []
for tx_hash in tqdm(token_transactions_df['Txhash'].unique().tolist()):
    all_tokens_df.append(extract_tokens_info(tx_hash))
all_tokens_df = pd.concat(all_tokens_df)
all_tokens_df.to_pickle("all_tokends_df")

100%|██████████| 6739/6739 [1:08:43<00:00,  1.63it/s]  


In [25]:
all_tokens_df[all_tokens_df['to'] == """AttributeError("'NoneType' object has no attribute 'text'")"""]

,from,to,amount,usd,token_hash,Txhash


In [31]:
all_tokens_df['amount'] = pd.to_numeric(all_tokens_df['amount'], errors='coerce')
all_tokens_df['usd'] = pd.to_numeric(all_tokens_df['usd'], errors='coerce')

In [34]:
all_tokens_df['usd'].isna().sum()

9510

In [35]:
all_tokens_df

,from,to,amount,usd,token_hash,Txhash
0,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Convex Finance: Voter Proxy,NaN,NaN,0x5282a4ef67d9c33135340fb3289cc1711c13638c,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
0,Convex Finance: Voter Proxy,Curve.fi: IronBank Liquidity Gauge,NaN,NaN,0x5282a4ef67d9c33135340fb3289cc1711c13638c,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
0,Null Address: 0x000…000,Convex Finance: Voter Proxy,NaN,NaN,0xf5194c3325202f456c95c1cf0ca36f8475c1949f,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
0,Null Address: 0x000…000,Convex Finance: Booster,NaN,NaN,0x912ec00eaebf3820a9b0ac7a5e15f381a1c91f22,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
0,Convex Finance: Booster,0x3e03fff82f77073cc590b656d42fceb12e4910a8,NaN,NaN,0x912ec00eaebf3820a9b0ac7a5e15f381a1c91f22,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,0xb550117ab3e407bc203ad24ae42a64a16971e833f9ee...
0,NaN,NaN,NaN,NaN,NaN,0x720f6a4d8999ae940cb2df3a633034ea9ca1078bef97...
0,Null Address: 0x000…000,Ren: RenBridge,99.899301,1162749.46,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,0x63f876ae7c626619a740f26d135dc187f2c1dd24d42f...
0,Null Address: 0x000…000,RenVM: Darknodes Fee renBTC,0.099999,1163.91,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,0x63f876ae7c626619a740f26d135dc187f2c1dd24d42f...


In [76]:
from fake_useragent import UserAgent
ua = UserAgent(use_cache_server=False)
# proxies = {"https": "188.170.233.113:3128"}
transaction_hash = "https://etherscan.io/tx/0x3f47026093d13668760dc35418cb4983b374e9ebdc5af2cca2d93559b83dd10f"
req = requests.get(f"https://etherscan.io/tx/{transaction_hash}",
                   headers={"User-Agent":ua.random},
                   # proxies=proxies
                   )
content = req.content
soup = BeautifulSoup(content)
all_tokens = soup.find_all("li", class_='media align-items-baseline mb-2')
token_url = all_tokens[0]
token1_amount = token_url.find("span", 'data-toggle'=='tooltip').text
amount = float(token1_amount.replace(",", "_"))

IndexError: list index out of range

In [77]:
soup

<!DOCTYPE html>
<html lang="en">
<head><title>Etherscan Error Page</title>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="The Ethereum BlockChain Explorer, API and Analytics Platform" name="Description"/>
<meta content="etherscan.io" name="author"/>
<meta content="ethereum, explorer, ether, search, blockchain, crypto, currency" name="keywords"/>
<meta content="telephone=no" name="format-detection"/>
<link href="/images/favicon2.ico" rel="shortcut icon"/>
<link href="/assets/vendor/font-awesome/css/fontawesome-all.min.css?v=0.11.4.0" rel="stylesheet"/>
<link href="/assets/css/theme.min.css?v=20.2.4.0" rel="stylesheet"/>
<script async="" src="/cdn-cgi/bm/cv/669835187/api.js"></script>
</head><body>
<header class="u-header" id="header">
<div class="u-header__section">
<div class="container" id="logoAndNav">
<nav class="js-mega-menu navbar navbar-expand-md u-header__navbar u-header__navbar--no-space 1d-block">
<div>
<a aria-label="Eth

In [59]:
all_tokens_df

,from,to,amount,usd,token_hash,Txhash
0,0x99fd1378ca799ed6772fe7bcdc9b30b389518962,Convex Finance: Voter Proxy,NaN,NaN,0x5282a4ef67d9c33135340fb3289cc1711c13638c,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
0,Convex Finance: Voter Proxy,Curve.fi: IronBank Liquidity Gauge,NaN,NaN,0x5282a4ef67d9c33135340fb3289cc1711c13638c,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
0,Null Address: 0x000…000,Convex Finance: Voter Proxy,NaN,NaN,0xf5194c3325202f456c95c1cf0ca36f8475c1949f,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
0,Null Address: 0x000…000,Convex Finance: Booster,NaN,NaN,0x912ec00eaebf3820a9b0ac7a5e15f381a1c91f22,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
0,Convex Finance: Booster,0x3e03fff82f77073cc590b656d42fceb12e4910a8,NaN,NaN,0x912ec00eaebf3820a9b0ac7a5e15f381a1c91f22,0x2394fdb6228d761f899156ebbc06554d0701c5952500...
...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,0xb550117ab3e407bc203ad24ae42a64a16971e833f9ee...
0,NaN,NaN,NaN,NaN,NaN,0x720f6a4d8999ae940cb2df3a633034ea9ca1078bef97...
0,Null Address: 0x000…000,Ren: RenBridge,99.899301,1162749.46,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,0x63f876ae7c626619a740f26d135dc187f2c1dd24d42f...
0,Null Address: 0x000…000,RenVM: Darknodes Fee renBTC,0.099999,1163.91,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,0x63f876ae7c626619a740f26d135dc187f2c1dd24d42f...


In [74]:
ua.random

'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0'